In [1]:
from pykafka import KafkaClient

In [2]:
client = KafkaClient(hosts="127.0.0.1:9092")

In [3]:
client.topics

{b'sample': <pykafka.topic.Topic at 0x7fcdcb56fa60 (name=b'sample')>,
 b'raw_recipes': <pykafka.topic.Topic at 0x7fcdcad3e610 (name=b'raw_recipes')>,
 b'fashion': <pykafka.topic.Topic at 0x7fcdcad6ee80 (name=b'fashion')>}

In [4]:
topic = client.topics['raw_recipes']

In [5]:
with topic.get_sync_producer() as producer:
    for i in range(4):
        producer.produce(str.encode('test message ' + str(i ** 2)))

In [6]:
consumer = topic.get_simple_consumer(consumer_timeout_ms=5000)
for message in consumer:
    if message is not None:
        print(message.offset, message.value)

0 b'a'
1 b'b'
2 b'c'
3 b'a'
4 b'b'
5 b'c'
6 b'test message 0'
7 b'test message 1'
8 b'test message 4'
9 b'test message 9'
10 b'test message 0'
11 b'test message 1'
12 b'test message 4'
13 b'test message 9'
14 b'test message 0'
15 b'test message 1'
16 b'test message 4'
17 b'test message 9'
18 b'test message 0'
19 b'test message 1'
20 b'test message 4'
21 b'test message 9'
22 b'test message 0'
23 b'test message 1'
24 b'test message 4'
25 b'test message 9'
26 b'test message 0'
27 b'test message 1'
28 b'test message 4'
29 b'test message 9'
30 b'test message 0'
31 b'test message 1'
32 b'test message 4'
33 b'test message 9'
34 b"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [57]:
from google.cloud import pubsub_v1

In [58]:
import os

In [59]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="fashionclass-1da73e1f76c2.json"

In [60]:
project_id = "fashionclass"
topic_id = "fashion"

In [61]:
publisher = pubsub_v1.PublisherClient()

In [62]:
topics = []
for topic in publisher.list_topics(request={"project": "projects/fashionclass"}):
    print(topic.name)
    topics.append(topic.name)

projects/fashionclass/topics/test
projects/fashionclass/topics/fashion


In [63]:
topic_path = publisher.topic_path(project_id, topic_id)

if topic_path not in topics:
    topic = publisher.create_topic(request={"name": topic_path})

In [64]:
from concurrent import futures

In [65]:
publish_future = publisher.publish("projects/fashionclass/topics/fashion", data=str.encode('you did it'))
future = futures.wait([publish_future], return_when=futures.ALL_COMPLETED)
print(publish_future.result(timeout=60))

4434009598453775


In [66]:
from google.cloud import pubsub

In [67]:
subscriber = pubsub.SubscriberClient()

In [68]:
sub_path = subscriber.subscription_path("fashionclass", "fashion_lover")

In [69]:
current_subs = []
for subscription in subscriber.list_subscriptions(request={"project": "projects/fashionclass"}):
    print(subscription.name)
    current_subs.append(subscription.name)

projects/fashionclass/subscriptions/fashionista
projects/fashionclass/subscriptions/fashion_lover


In [70]:
if "projects/fashionclass/subscriptions/fashion_lover" not in current_subs:
    subscriber.create_subscription(request={"name": sub_path, "topic": "projects/fashionclass/topics/fashion"})

In [71]:
response = subscriber.pull(
    request={
        "subscription": sub_path,
        "max_messages": 5,
    }
)

In [72]:
for msg in response.received_messages:
    print("Received message:", msg.message.data)

Received message: b'you did it'


In [73]:
ack_ids = [msg.ack_id for msg in response.received_messages]
subscriber.acknowledge(
    request={
        "subscription": sub_path,
        "ack_ids": ack_ids,
    }
)

In [74]:
subscriber.close()